In [1]:
import os
import sys
notebook_path = os.getcwd()  # Get the current working directory
project_root = os.path.dirname(notebook_path)  # Get the parent directory
print(project_root)
print(notebook_path)
sys.path.append(project_root)
os.chdir(project_root)
sys.path.append(project_root + '/src')
sys.path = list(set(sys.path))


/home/alice/git-repo/task-extraction
/home/alice/git-repo/task-extraction/scripts
['', '/home/alice/git-repo/task-extraction', '/usr/lib/python3.10', '/home/alice/git-repo/task-extraction/src', '/home/alice/git-repo/task-extraction/venv/lib/python3.10/site-packages', '/usr/lib/python310.zip', '/usr/lib/python3.10/lib-dynload']


In [2]:
import json
from src.lb_annotation_utils import Article
from src.llm_utils import system_prompts, estimate_token_cost
import re
from openai import OpenAI
client = OpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=os.getenv("OPENROUTER_API_KEY"))

4e-05


In [3]:
# Define constants
MAP_FOLDER = "data/neurovault_labeled_papers/"
INDEX_FILE = MAP_FOLDER + "nv_labeled_tasks_pmcid_map.json"
TASK_FILE_PATTERN = MAP_FOLDER + "nv_labeled_tasks_{chunk_number}.json"

In [4]:
ann = json.load(open('data/labelbuddy_annotations/nv_collection/annotations.json'))

In [5]:
# Set up filter logic
# exclusion criteria
exclude_types = ["resting_state", "meta-analysis"]
# excluded articles
excluded_articles = []
# included articles
included_articles = []  
# extract methods
for i in range(len(ann)):
    article_obj  = Article(ann[i])
    if article_obj.type in exclude_types:
        excluded_articles.append({"pmcid": article_obj.pmcid, "exclusion_reason":{"type": article_obj.type}})
    elif article_obj.methods == "No explicit methods section":
        excluded_articles.append({"pmcid": article_obj.pmcid, "exclusion_reason":{"methods": "No explicit methods section"}})
    else:
        included_articles.append(article_obj)


No subheaders found in body section


In [6]:
excluded_articles

[{'pmcid': 5324609,
  'exclusion_reason': {'methods': 'No explicit methods section'}},
 {'pmcid': 5090046, 'exclusion_reason': {'type': 'resting_state'}},
 {'pmcid': 10634720, 'exclusion_reason': {'type': 'meta-analysis'}}]

In [7]:
included_articles

In [8]:
total_cost = 0
for i in range(len(included_articles)):
    article_obj  = included_articles[i]
    if article_obj.methods!="No explicit methods section":
        item = {"pmcid": article_obj.pmcid, "methods": article_obj.methods}
        total_cost += estimate_token_cost(article_obj.methods, "text-embedding-3-small")
    else:
        item = {"pmcid": article_obj.pmcid, "methods": None}
print(total_cost)


0.0024749999999999998


In [9]:
def extract_using_llm(text:str, client:OpenAI, call_message:callable):
    messages = call_message(text)
    response = client.chat.completions.create(
        model = 'openai/gpt-4o-mini',
        messages = messages,
        seed = 42, 
        response_format = {"type": "json_object"}
    )
    return response.choices[0].message.content
def extract_cognitive_task(text:str):
    messages = [
        {"role": "system", "content": f"{system_prompts['CognitiveTask']}"},
    {"role": "user", "content": f"{text}"},
]
    return messages
def extract_cognitive_task_description(text:str):
    messages = [
        {"role": "system", "content": f"{system_prompts['CognitiveTaskDescription']}"},
    {"role": "user", "content": f"{text}"},
]   
    return messages



In [10]:
for i in range(len(included_articles)):
    methods = included_articles[i].methods
    output = extract_using_llm(text = methods, client = client, call_message = extract_cognitive_task)
    if re.search(r'null', output):
        output_description = extract_using_llm(text = methods, client = client, call_message = extract_cognitive_task_description)
        included_articles[i].llm.cognitive_task_description = output_description
    else:
        included_articles[i].llm.cognitive_task = output


In [11]:
for i in range(len(included_articles)):
    print(included_articles[i].pmcid)
    print(included_articles[i].llm.cognitive_task)
    print(included_articles[i].llm.cognitive_task_description)
    print("\n")

8318202
{"cognitive_task": ["attentive listening", "word repetition"]}
None


9202476
{"cognitive_task": ["match-to-sample (MTS) task", "visual search", "visual attention", "reaction time measurement"]}
None




In [37]:
def get_nv_labeled_tasks(pmcid, index_file = INDEX_FILE,json_file_pattern = TASK_FILE_PATTERN):
    nv_labeled_tasks = []
    query_pmcid = str(pmcid)
    with open(index_file, "r") as f:
        indexer = json.load(f)
        task_location = indexer[query_pmcid]
        print([query_pmcid, task_location])
        for file in task_location:
            with open(file, "r") as ff:
                extracted_tasks = json.load(ff)
                for task in extracted_tasks:
                    if list(task.keys())[0] == query_pmcid:
                        nv_labeled_tasks.append(task)
    return nv_labeled_tasks
# Table

print(included_articles[1].pmid)
print(included_articles[1].doi)
nv_labeled_tasks = get_nv_labeled_tasks(pmcid = included_articles[1].pmid)
print(included_articles[0].llm.__dict__)
print(nv_labeled_tasks)

nv_labeled_tasks = get_nv_labeled_tasks(pmcid = included_articles[0].pmid)
print(included_articles[0].llm.__dict__)
print(nv_labeled_tasks)

35720693
10.3389/fnins.2022.850372
['35720693', ['data/neurovault_labeled_papers/nv_labeled_tasks_0.json']]
{'cognitive_task': '{"cognitive_task": ["attentive listening", "word repetition"]}', 'cognitive_task_description': None}
[{'35720693': {'pmcid': '35720693', 'task': {'id': 'trm_4c8a834779883', 'name': 'rest eyes open', 'definition_text': 'Subjects rest passively with their eyes open. Often used as a baseline for comparison for other tasks.'}, 'pmid': '35720693', 'doi': '10.3389/fnins.2022.850372'}}]
['34327333', ['data/neurovault_labeled_papers/nv_labeled_tasks_5.json']]
{'cognitive_task': '{"cognitive_task": ["attentive listening", "word repetition"]}', 'cognitive_task_description': None}
[{'34327333': {'pmcid': '34327333', 'task': {'id': 'trm_550b54a8b30f4', 'name': 'language processing fMRI task paradigm', 'definition_text': 'This task was developed by Binder and colleagues (Binder et al. 2011) and uses the E-prime scripts provided by these investigators. The task consists of 